examples

In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

from face_detections import face_detect, gray_scale, cv_face_detect

In [ ]:
os.path.dirname(os.path.abspath('__file__'))

In [ ]:
class CFG:
    data_dir = os.getcwd() + '/../input/data/train' # /opt/ml/project/DataProcessing/../input/data/train
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'

In [ ]:
num2class = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']
class2num = {k:v for v, k in enumerate(num2class)}
print(class2num)

df = pd.read_csv(CFG.df_path)
df.head()
df['path']

In [ ]:
def image_path(idx, normal=6):
    return CFG.img_dir +'/' + df['path'][idx] + '/' + num2class[normal] + '.jpg'

print(image_path(0, 0))
image = cv2.imread(image_path(0, 5))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
image = cv2.imread(image_path(1246, 2))

face_image, x, y= face_detect(image, gray_scale)
# if face_image is None:
#     face_image, x, y = cv_face_detect(image)
# face_image = cv_face_detect(image)
print(face_image.shape)
print(x, y)    
plt.imshow(face_image)
plt.show()
print(face_image.shape)

In [ ]:
image = cv2.imread(image_path(132, 5))
h, w, c = image.shape
h = 195
w = 176
x = 102
y = 193
image = image[y:y+h, x:x+w]
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# no image: 006056_female_Asian_19/mask2.jpg -> 006056_femail_Asian_19/mask2.png
# print(image_path(2256, 2))
# image = cv2.imread(image_path(2256, 2))
# print(image is None)

image = cv2.imread(image_path(254, 6))
face_image, x, y = cv_face_detect(image)
print(x, y) # [x, y, x+w, y+h] # faces[1]:faces[3], faces[0]:faces[2])
print(face_image.shape)
plt.imshow(face_image)
plt.show()



In [ ]:
heights = []
widths  = []
x_list = []
y_list = []
count = 0
no_image_paths = []
no_face_detect = []
for i in range(len(df['path'])):
    for j in range(len(num2class)):
        image = cv2.imread(image_path(i, j))
        if image is None:
            no_image_paths.append(image_path(i, j))
            continue
        face_image, x, y = face_detect(image, gray_scale)
        if face_image is None:
            face_image, x, y = cv_face_detect(image)
        if face_image is not None:
        # face_image = cv_face_detect(image)
            h, w, _ = face_image.shape
            heights.append(h)
            widths.append(w)
            x_list.append(x)
            y_list.append(y)
        else:
            count += 1
            no_face_detect.append(image_path(i, j))
            

In [ ]:
print(round(sum(heights) / len(heights))) # 195
print(round(sum(widths) / len(widths))) # 176
print(round(sum(x_list) / len(x_list))) # 102
print(round(sum(y_list) / len(y_list))) # 193
print(len(heights), len(widths))
print(f'no image path detected: {len(no_image_paths)}')
print(f'no face detected: {count}')

'''
mean height: 195
mean width: 176
num of images: 17816 17816
no image path detected: 865 -> *.png, *.jpeg
no face detected: 219
'''